In [1]:
from libraries import *
from parameters import *
import statsmodels.api as sm
import sklearn.feature_selection
import statsmodels.api as sm
import sklearn.feature_selection
from sklearn.preprocessing import PolynomialFeatures
import warnings
warnings.filterwarnings('ignore')
#from statannot import add_stat_annotation
from numpy.random import seed
from numpy.random import randint
import statsmodels.formula.api as smf
import random

In [2]:
os.getcwd()
os.chdir(projectDir)

In [3]:
myGuide=0

In [ ]:
# adataC = sc.read('outputs/anndata/adata-hash-features_singlets_SingleKO_05242020_Filtered.h5ad')
# controlGuideCols = adataC.obs.columns[[ (x.startswith("ONE_NONGENE_SITE") or x.startswith("NO_TARGET")) for x in adataC.obs.columns]]
# fBarMat = adataC.obs[controlGuideCols]
# fBarMat[fBarMat>0] = 1
# adataControl = adataC[fBarMat.sum(axis=1) > 0,:]
# adataControl.write("outputs/anndata/adata-hash-features_singlets_SingleKO_05242020_Filtered_ControlCellsOnly.h5ad")

In [4]:
adataControl = sc.read("outputs/anndata/adata-hash-features_singlets_SingleKO_05242020_Filtered_ControlCellsOnly.h5ad")


In [5]:
adataControl.X.shape

(41123, 6685)

In [6]:
controlGuideCols = adataControl.obs.columns[[ (x.startswith("ONE_NONGENE_SITE") or x.startswith("NO_TARGET")) for x in adataControl.obs.columns]]

In [7]:
expressionMatrix = adataControl.X

In [8]:
geneCol = controlGuideCols[[myGuide]].tolist() + ["n_genes", "mt_frac", "leiden"]
guideMatrix = adataControl.obs[geneCol]
allVars = pd.Series(geneCol) 
all_columns = "+".join(allVars[~allVars.isin(["leiden"])])
my_formula = "y~" + all_columns
print(my_formula)


y~NO_TARGET_1+n_genes+mt_frac


In [ ]:

#geneCol = controlGuideCols[myGuide].tolist() + ["n_genes", "mt_frac", "leiden"]

for i in range(0,6680,80):
    print(i)
    pvalDF = pd.DataFrame()
    coefDF = pd.DataFrame()

    for j in range(i,i+80):
        print(j)
        guideMatrix["y"] = np.array(expressionMatrix)[:,j]
        est = smf.mixedlm(my_formula, guideMatrix, groups=guideMatrix["leiden"], missing='drop')
        mdf = est.fit(method=["bfgs"])
        pvalDF[str(j)] = mdf.summary().tables[1]["P>|z|"]
        coefDF[str(j)] = mdf.summary().tables[1]["Coef."]

    coefDF.index = mdf.summary().tables[1].index
    pvalDF.index = mdf.summary().tables[1].index
    pvalDF.to_csv("MixedEffectControlLMPerGuideOutputs/pvalues_"+str(i)+"_"+str(i+80)+"_Control_MixedEffect_"+str(myGuide)+".csv")   
    coefDF.to_csv("MixedEffectControlLMPerGuideOutputs/coefs_"+str(i)+"_"+str(i+80)+"_Control_MixedEffect_"+str(myGuide)+".csv")   